## Extract Diagnoseses, Symptoms, and Procedures using GPT-4o

**Goal of this notebook:**  
Use AI to extract ONLY the important components of the Mimic-III NOTEEVENT TEXT field. Removing details such as admission dates and medical dosages will reduce the token count, and simplify the text _while_ keeping relevant context and the original sentences that would be lost if we were to use a parser such as [Azure Text Analytics for Health](./01_az_text_analytics.ipynb) or the [mednlp](https://github.com/plandes/mednlp) python package.

The extracted data will be used to...  
- Simplify the note text before attempting to code. This will improve calssification results

**Requirements**  
- Setup Azure Open AI Resource with GPT-4o deployment, ensure [.env](./.env.sample) file is populated up to date

In [35]:
from src.prepare_mimic_iii import transform_data
from dotenv import load_dotenv, find_dotenv
from openai import AzureOpenAI
from textwrap import dedent

import pandas as pd
import os
import tiktoken

load_dotenv(find_dotenv(), override=True)

pd.set_option('display.max_colwidth', None)

#### Prepare Data

In [27]:
# Get medical coding data, take a small subsample

# df = transform_data("data/") # Only re-run if change in preparation logic
df = pd.read_csv("data/joined/dataset_single_001_088.csv.gz").sample(5, replace=False,random_state=1234)
print(df.shape)
display(df.dtypes)
display(df.head(5))

(5, 3)


HADM_ID       int64
TEXT         object
ICD9_CODE    object
dtype: object

HADM_ID  \
4808   199105   
836    116802   
4510   192358   
1574   131932   
4533   192984   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

#### Use AOAI to Simplify Note Text

In [28]:
aoai_client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_BASE"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    api_version="2024-02-01"
)

In [33]:
# Function to get token counts

def get_token_counts(text):
    encoding = tiktoken.get_encoding('cl100k_base')
    num_tokens = len(encoding.encode(text))
    return num_tokens

In [37]:
# Funciton to build prompt

def build_prompt(note):
    sys = """
    Parse the following medical note. Return any sentences that relate to a diagnosis or symptom. Ignore other information such as patient name, dates, medicine types, dosage amounts, etc.
    DO NOT ADD ANY INFORMATION TO THE NOTE. ONLY RETURN THE RELEVANT SENTENCES. 
    """
    prompt = f"{note}"

    return (sys, prompt)

In [38]:
def aoai_extract(note):
    sys, prompt = build_prompt(note)
    response = aoai_client.chat.completions.create(
        model=os.getenv("AOAI_MAIN_DEPLOYMENT_NAME"), # model = "deployment_name".
        messages=[
            {"role": "system", "content": dedent(sys)},
            {"role": "user", "content": dedent(prompt)}
        ],
    )

    return response.choices[0].message.content


In [39]:
df["AOAI_EX"] = df["TEXT"].apply(lambda x: aoai_extract(x))

INFO:httpx:HTTP Request: POST https://medcode-aoai-useast.openai.azure.com//openai/deployments/gpt-4o-deploy/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-useast.openai.azure.com//openai/deployments/gpt-4o-deploy/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-useast.openai.azure.com//openai/deployments/gpt-4o-deploy/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-useast.openai.azure.com//openai/deployments/gpt-4o-deploy/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://medcode-aoai-useast.openai.azure.com//openai/deployments/gpt-4o-deploy/chat/completions?api-version=2024-02-01 "HTTP/1.1 200 OK"


In [74]:
df["AOAI_TOKENS"] = df["AOAI_EX"].apply(lambda x: get_token_counts(x))
df["TEXT_TOKENS"] = df["TEXT"].apply(lambda x: get_token_counts(x))

#### Examine Results

In [44]:
# Avg token count difference - How much efficiency is gained?
print(f"Average Original Text tokens: {df['TEXT_TOKENS'].mean()}")
print(f"Average AOAI Extracted tokens: {df['AOAI_TOKENS'].mean()}")
print(f"Average token count difference: {(df['TEXT_TOKENS'] - df['AOAI_TOKENS']).mean()}")

Average Original Text tokens: 1332.2
Average AOAI Extracted tokens: 457.8
Average token count difference: 874.4


In [66]:
# Text analysis - Is any information lost?
sub_df=df[['ICD9_CODE', 'AOAI_EX', 'AOAI_TOKENS', 'TEXT_TOKENS']]

In [67]:
# Example 0
display(sub_df.iloc[[0]])

,ICD9_CODE,AOAI_EX,AOAI_TOKENS,TEXT_TOKENS
4808,['041'],"- Chief Complaint Respiratory distress\n- History Present Illness Mr . Known lastname year old male complex medical history originally admitted hospital Month strangulated ventral hernia .\n- Secondary respiratory distress/ ? pneumonia well management volume overload .\n- CAD s/p LAD OM stent , s/p RCA PTCA .\n- DM .\n- HTN .\n- PVD s/p bilat LE bypass surgeries Dr.Last Name STitle .\n- CRI baseline Cr . .\n- Cataracts .\n- Gout .\n- BPH .\n- Abd hernia .\n- s/p CCY , ex lap w/abd hernia resulting .\n- Incarcerated ventral hernia containing strangulated small bowel requiring small bowel resection .\n- Complicated leak leading operation .\n- Hx Etoh abuse x yrs , quit .\n- Fa died secondary colon ca Mo died secondary PNA Siblings Etoh abuse , HTN \n- Breath sounds course bilaterally , wheezes rhonchi\n- + upper & lower extremity edema\n- BLOOD WBC . RBC . Hgb . Hct . MCV MCH . MCHC . RDW . Plt Ct\n- SPARSE GROWTH OROPHARYNGEAL FLORA .\n- STAPH AUREUS COAG + .\n- Oxacillin RESISTANT Staphylococci MUST reported also RESISTANT penicillins , cephalosporins , carbacephems , carbapenems , beta lactamase inhibitor combinations . Rifampin used alone therapy .\n- STAPH AUREUS COAG + | CLINDAMYCIN = > R ERYTHROMYCIN = > R GENTAMICIN < = . LEVOFLOXACIN = > R OXACILLIN = > R PENICILLIN = > . R RIFAMPIN < = . TETRACYCLINE VANCOMYCIN < =\n- FUNGUS ISOLATED .\n- FINAL REPORT AEROBIC BOTTLE Final GROWTH .\n- STAPH AUREUS COAG + .\n- FINAL SENSITIVITIES .\n- Oxacillin RESISTANT Staphylococci MUST reported also RESISTANT penicillins , cephalosporins , carbacephems , carbapenems , beta lactamase inhibitor combinations . Rifampin used alone therapy .\n- VANCOMYCIN SENSITIVITY DONE E TEST ..\n- STAPH AUREUS COAG + | CLINDAMYCIN = > R ERYTHROMYCIN = > R GENTAMICIN < = . LEVOFLOXACIN = > R OXACILLIN = > R PENICILLIN = > . R RIFAMPIN < = . TETRACYCLINE VANCOMYCIN \n- Mr . Known lastname readmitted surgical intensive care unit Hospital rehabilitation facility .\n- Diuresis initiated good response , weaned ventilator placed tracheostomy mask ventilation .\n- Initially well , noted large cuff leak tracheostomy , became somewhat tachypneic .\n- Placed back ventilator responded well .\n- Noted fever blood/urine cultures sent .\n- Found staphylococcus sputum blood , pseudomonas urine .\n- Appropriate antibiotic coverage initiated linezolid cefepime , defervesced , corresponding drop WBC .\n- Tracheostomy tube removed replaced similarly sized flexible Last Name un tracheostomy , angled specifically longer arm .\n- Air leak resolved maneuver .\n- Respiratory distress .\n- Congestive heart failure .\n- Air leak tracheostomy .\n- Pancreatitis .\n- Pneumonia .\n- Incarcerated strangulated ventral hernia , Small bowel resection primary reanastomosis , multiple abdominal abscesses , respiratory failure , myocardial infarction .\n- CAD s/p LAD OM stent , s/p RCA PTCA .\n- DM .\n- HTN .\n- PVD .\n- CRI Cr . .\n- cataracts .\n- gout .\n- BPH .\n- h/o EtOH abuse quit yrs ago , h/o heavy tobacco use",861,1832


Analysis:  
- Code 041 = "Bacterial infection in conditions classified elsewhere"
- The important component in this comment is the mention of 'staphylococcus' which is not lost despite a near 1000 token reduction

---


In [71]:
# Example 1
display(sub_df.iloc[[1]])

,ICD9_CODE,AOAI_EX,AOAI_TOKENS,TEXT_TOKENS
836,['041'],Atrial fibrillation. Right bundle branch block. Non specific ST wave changes. Compared previous tracing significant change.,22,24


Analysis:  
- Code 041 = "Bacterial infection in conditions classified elsewhere"
- This comment was unchanged

---

In [72]:
# Example 2
display(sub_df.iloc[[2]])

,ICD9_CODE,AOAI_EX,AOAI_TOKENS,TEXT_TOKENS
4510,['039'],"- Chief Complaint AMS \n- History Present Illness yo F IPF, COPD L chronic prednisone, CHF, mechanical mitral valve, s/p pacemaker placement, known high grade colonic adenoma GIB resected, gastric varix history liver disease recently admitted nocardia pneuomina, presented AMS.\n- Patient transferred MICU given + melena likely need endoscopy may need intubation airway protection.\n- Patient denies complaints shortness breath, typical.\n- Rectal heme postive dark stool.\n- Labs notable WBC count, Hct, creatinine.\n- baseline, metabolic alkylosis VBG chronic.\n- crossed four units. \n- ROS reported wanting sleep.\n- Increased frequency BMs recently describes dark, bloody. recall sticky. \n- Denies fevers, chills, chest pain, shortness breath, abdominal pain, nausea, vomiting, anything else.\n- s/p mechanical mitral valve sinus node dysfunction s/p DDD pacemaker placement atrial flutter s/p ablation / cardioversion congestive heart failure, Last Echo, mildly depressed LVEF= % systolic function chronic obstructive pulmonary disease LO trach home rest idiopathic pulmonary fibrosis chronic prednisone chronic kidney disease baseline creatinine . . anemia due mechanical valve chronic kidney disease hypertension hypercholesterolemia hypothyroidism meniere ? ? ? ? ? ? disease HOH spinal arthritis breast cancer radical mastectomy right breast. Partial left. s/p hysterectomy s/p nasal embolization refractory epistaxis lower GI bleed secondary high grade colonic adenoma s/p biopsy resection\n- General lethargic, oriented person place, trying pull things\n- Lungs Clear auscultation bilaterally anteriorly, wheezes ronchi\n- Vitals / . / % L General alert, aao, sitting bed\n- Lungs improved wheezing.\n- colonic adenoma Hct drop small melena, s/p transfusions appropriate Hct bump, likely chronic bleeding colonic adenoma.\n- Patient escalation care, treat acute blood loss transfusion supportive care.\n- patient family decided stop anticoagulation.\n- Melena/Dysplastic Adenoma ED, initial hct, rose unit PRBC.\n- Patient known malignant colonic mass gastric varices.\n- colonic mass proven malignant biopsy \n- drop Hct large melanotic BM.\n- concern thrombocytopenia, octreotide stopped despite continued slow bleeding.\n- Patient transferred medicine floor.\n- Patient one episode point hematocrit drop, bumped two units packed red cells.\n- Nocardia PNA Diagnosed BAL previous admission.\n- Altered mental status Mental status waxed waned, sometimes confused, generally oriented person, place year.\n- Systolic congestive heart failure, chronic Per Echo, mildly depressed LVEF= % systolic function.\n- appear volume overloaded exam presently.\n- Mechanical MVR Valve Anticoagulation held indefinitely lengthy discussion patient HCP\n- COPD/IPF continued nebs steroids.\n- HYPOTHYROIDISM continued home levothyroxine.\n- HYPERTENSION restarted bumex, holding metolazone.\n- gastrointestinal bleeding colonic mass nocardia pneumonia acute chronic kidney failure\n- Mental Status Confused sometimes.\n- admitted hospital acute gastrointestinal bleed. \n- likely bleeding originated colon, known mass.\n- Also, history esophageal varices may bleeding well.\n- also treated pneumonia antibiotics, continue follow infectious disease physician outpatient.",726,2963


Analysis:  
- Code 039 = "Actinomycotic infections"
- TODO...

---

In [70]:
# Example 3
display(sub_df.iloc[[3]])

,ICD9_CODE,AOAI_EX,AOAI_TOKENS,TEXT_TOKENS
1574,"['042', '070']","- Admitting Diagnosis UNSTABLE ANGINA\n- Hospital MEDICAL CONDITION year old man s/p emergency AVR/MV repair/cabg x/repl . asc . aorta/ fem . bypass/cardiogenic shock\n- LOW LUNG VOLUMES\n- EXTENT PULMONARY EDEMA UNCHANGED\n- SUGGESTION OPEN CHEST WALL\n- HISTORY Emergency valve replacements\n- IMPRESSION AP chest compared intraoperative study a.m . Moderately severe pulmonary edema improved , though , though lung volumes remain quite low\n- Mediastinum normal postoperative appearance\n- Given delayed sternal closure persistently low lung volumes\n- pneumothorax pleural effusion , , small",148,379


Analysis:  
- Code 042 = "HIV" ; Code 070 = "Viral Hepatitis"
- TODO

---

In [73]:
# Example 4
display(sub_df.iloc[[4]])

,ICD9_CODE,AOAI_EX,AOAI_TOKENS,TEXT_TOKENS
4533,"['041', '038']","- Chief Complaint Respiratory Distress\n- Major Surgical Invasive Procedure PEG tube exchange\n- History Present Illness y/o F w/ dementia, non verbal p/w respiratory distress diarrhea.\n- Pt. recently treated recurrent respiratory infection w/ augmentin developed diarrhea.\n- diarrhea several days today son noted respiratory distress brought Location un ED.\n- intubated discussion w/ son goals care transferred Hospital ED started levophed.\n- SBP. got L NS, cefepime vanc IV.\n- Location un hyperkalemia w/o EKG changes got insulin, D.\n- Past Medical History Dementia, nonverbal baseline Left hip decubitus ulcer Sacral decubitus ulcer diabetes urinary retention CVA years ago Recurrent pulmonary infections Old necrotic left great toe\n- Unresponsive stimuli, vent levophed drip. mcg/min.\n- large decubitus ulcer/burn wound L hip. L foot w/ necrotic great toe.\n- Pertinent Results URINE Source Catheter. FINAL REPORT URINE CULTURE Final PSEUDOMONAS AERUGINOSA.\n- PSEUDOMONAS AERUGINOSA MODERATE GROWTH. THREE COLONIAL MORPHOLOGIES. YEAST. MODERATE GROWTH.\n- MRSA SCREEN Source Nasal swab. FINAL REPORT MRSA SCREEN Final POSITIVE METHICILLIN RESISTANT STAPH AUREUS.\n- BLOOD WBC.\n- BLOOD Lactate.\n- Brief Hospital Course Sepsis Ms Known lastname hypotensive hypoxic admission intubated required pressors.\n- Lactate treated empirically hospital acquired pneumonia cefepime vancomycin narrowed cefepime sputum cultures returned positive pseudomonas aeruginosa.\n- UA positive culture also grew pseudomonas.\n- concern c. diff colitis initially treated PO vancomycin, discontinued toxin assays returned negative.\n- sources infection considered included various decubitus ulcers, burn related ulcer, necrotic great toe.\n- Left upper extremity DVT Ms. Known lastname noted edematous left arm hospitalization setting left internal jugular catheter.\n- Ultrasound demonstrated cephalic vein DVT.\n- Anemia Hct trended, stabilized, hospitalization.\n- DM Ms. Known lastname managed ISS, decreased HD following episode hypoglycemia.\n- Goals care Ms. Known lastname eventually made DNR/DNI discussions sons, discharged home hospice services.\n- Sepsis\n- Respiratory failure\n- Dementia\n- Acute renal failure",532,1463


Analysis:  
- Code 041 = "Bacterial infection in conditions classified elsewhere" ; Code 038 = "Septicaemia"
- TODO

---